In [1]:
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

import pandas as pd
pd.options.display.max_columns = 100

from matplotlib import pyplot as plt
import numpy as np

import seaborn as sns
sns.set(rc={'figure.figsize':(12,9)})
import pylab as plot
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss,accuracy_score

In [2]:
from xgboost import XGBClassifier
import xgboost as xgb

In [3]:
#for scaling
from sklearn.preprocessing import StandardScaler

Importing training dataset

In [4]:
data = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')


In [5]:
print(data.shape)

(867873, 10)


In [6]:
data.columns

Index(['Dates', 'Category', 'Descript', 'DayOfWeek', 'PdDistrict',
       'Resolution', 'Address', 'X', 'Y', 'Id'],
      dtype='object')


Splitting the "Dates" into date and time

In [7]:
df=data
datetime=df.Dates.str.split(pat=" ",expand=True)
datetime.columns=['Date','Time']
#datetime

#------------test data-------------

df_test=test
datetime_test=df_test.Dates.str.split(pat=" ",expand=True)
datetime_test.columns=['Date','Time']

In [8]:
Date=datetime.Date.str.split(pat="-",expand=True)
Date.columns=['Year','Month','Day']

Time=datetime.Time.str.split(pat=":",expand=True)
Time.columns=['Hour','Minute','Second']

#------------test data-------------

Date_test=datetime_test.Date.str.split(pat="-",expand=True)
Date_test.columns=['Year','Month','Day']

Time_test=datetime_test.Time.str.split(pat=":",expand=True)
Time_test.columns=['Hour','Minute','Second']

Dropping the column of "Dates" and appending the dataframe "datetime"

In [9]:
df=pd.concat([df,Date,Time],axis=1)
#df

#-----------test data------------------

df_test=pd.concat([df_test,Date_test,Time_test],axis=1)


In [10]:
df=df.drop(labels=['Dates'],axis=1)

#-----------test data------------------

df_test=df_test.drop(labels=['Dates'],axis=1)


# #Encoding

In [11]:
df.columns

Index(['Category', 'Descript', 'DayOfWeek', 'PdDistrict', 'Resolution',
       'Address', 'X', 'Y', 'Id', 'Year', 'Month', 'Day', 'Hour', 'Minute',
       'Second'],
      dtype='object')

LabelEncoding of "DayOfWeek" and "PdDistrict"

In [12]:
le = preprocessing.LabelEncoder()

In [13]:
#y=pd.get_dummies(df.Category,columns=['Category'],prefix=" ",prefix_sep=" ",drop_first=True,)

le_res=le.fit_transform(df['Category'])
y=pd.DataFrame(le_res)
y.columns=['Category']
#y

# Rotation 60 degree

In [14]:
df["rot60_X"]=(0.25881) * df["Y"] + (0.96592) * df["X"]
df["rot60_Y"]=0.25881 * df["Y"] - (0.96592) * df["X"]


df_test["rot60_X"]=(0.25881) * df_test["Y"] + (0.96592) * df_test["X"]
df_test["rot60_Y"]=0.25881 * df_test["Y"] - (0.96592) * df_test["X"]

In [15]:
df["radial60"]=np.sqrt(np.power(df['rot60_X'],2) + np.power(df['rot60_Y'],2))

df_test["radial60"]=np.sqrt(np.power(df_test['rot60_X'],2) + np.power(df_test['rot60_Y'],2))

# Month clubbing

In [16]:
df


,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,Id,Year,Month,Day,Hour,Minute,Second,rot60_X,rot60_Y,radial60
0,SEX OFFENSES FORCIBLE,"FORCIBLE RAPE, BODILY FORCE",Friday,MISSION,NONE,2100 Block of MISSION ST,-122.419331,37.762264,141546,2013,06,28,17,40,00,-108.474029,128.020532,167.797114
1,LIQUOR LAWS,CONSUMING ALCOHOL IN PUBLIC VIEW,Thursday,SOUTHERN,"ARREST, BOOKED",1000 Block of MARKET ST,-122.411340,37.781271,794152,2004,02,19,02,46,00,-108.461390,128.017732,167.786808
2,FRAUD,"CREDIT CARD, THEFT BY USE OF",Wednesday,SOUTHERN,NONE,800 Block of BRYANT ST,-122.403405,37.775421,531205,2007,11,14,00,01,00,-108.455240,128.008553,167.775829
3,ROBBERY,ROBBERY OF A CHAIN STORE WITH A GUN,Thursday,BAYVIEW,DISTRICT ATTORNEY REFUSES TO PROSECUTE,2400 Block of SAN BRUNO AV,-122.404715,37.730161,523137,2007,12,27,18,30,00,-108.468220,127.998106,167.776249
4,OTHER OFFENSES,PROBATION VIOLATION,Sunday,SOUTHERN,"ARREST, BOOKED",4TH ST / STEVENSON ST,-122.405239,37.785265,200968,2012,09,09,17,02,00,-108.454464,128.012873,167.778623
5,WARRANTS,ENROUTE TO DEPARTMENT OF CORRECTIONS,Wednesday,MISSION,"ARREST, BOOKED",15TH ST / NATOMA ST,-122.418405,37.766760,75749,2014,05,14,06,14,00,-108.471970,128.020800,167.795988
6,BURGLARY,"BURGLARY, UNLAWFUL ENTRY",Wednesday,CENTRAL,"ARREST, BOOKED",0 Block of MARK LN,-122.404651,37.790442,617521,2006,08,09,23,45,00,-108.452556,128.013644,167.777979
7,SUSPICIOUS OCC,SUSPICIOUS OCCURRENCE,Friday,INGLESIDE,NONE,100 Block of PUTNAM ST,-122.410871,37.737526,186953,2012,11,16,12,41,00,-108.472259,128.005957,167.784851
8,OTHER OFFENSES,"DRIVERS LICENSE, SUSPENDED OR REVOKED",Monday,BAYVIEW,"ARREST, CITED",HALE ST / SAN BRUNO AV,-122.405911,37.732964,584559,2007,02,05,23,53,00,-108.468649,127.999986,167.777961
9,DRUG/NARCOTIC,UNDER THE INFLUENCE OF CONTROLLED SUBSTANCES,Tuesday,NORTHERN,"ARREST, BOOKED",ELLIS ST / LARKIN ST,-122.417710,37.784236,653619,2006,02,07,07,41,00,-108.466777,128.024653,167.795570


In [17]:
df_test

,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,Id,Year,Month,Day,Hour,Minute,Second,rot60_X,rot60_Y,radial60
0,ATTEMPTED GRAND THEFT FROM PERSON,Sunday,TENDERLOIN,NONE,100 Block of POWELL ST,-122.407878,37.785968,349598,2010,07,04,23,00,00,-108.456831,128.015604,167.782237
1,BATTERY OF A POLICE OFFICER,Saturday,PARK,"ARREST, BOOKED",2400 Block of GEARY BL,-122.443597,37.782644,766313,2004,06,26,22,43,00,-108.492193,128.049245,167.830764
2,VIOLATION OF MUNICIPAL CODE,Saturday,SOUTHERN,"ARREST, CITED",HOWARD ST / 3RD ST,-122.400474,37.785029,169887,2013,02,09,21,26,00,-108.449922,128.008209,167.772129
3,POSS OF PROHIBITED WEAPON,Sunday,NORTHERN,"ARREST, CITED",VANNESS AV / HAYES ST,-122.419698,37.777301,594704,2006,12,03,22,30,00,-108.470491,128.024778,167.798067
4,PETTY THEFT FROM UNLOCKED AUTO,Sunday,PARK,NONE,BEULAH ST / SHRADER ST,-122.451488,37.767516,47900,2014,09,21,08,45,00,-108.503730,128.052952,167.841050
5,LOST PROPERTY,Sunday,TENDERLOIN,NONE,EDDY ST / JONES ST,-122.412597,37.783932,339260,2010,08,29,18,00,00,-108.461917,128.019636,167.788600
6,INFLICT INJURY ON COHABITEE,Sunday,NORTHERN,NONE,100 Block of HAIGHT ST,-122.424641,37.772789,169575,2013,02,10,17,00,00,-108.476434,128.028385,167.804660
7,"VEHICLE, RECOVERED, AUTO",Monday,TARAVAL,NONE,TAPIA DR / PINTO AV,-122.481353,37.720399,681483,2005,09,19,16,00,00,-108.544772,128.069605,167.880289
8,"CHECKS, MAKE OR PASS FICTITIOUS",Tuesday,NORTHERN,NONE,200 Block of FELL ST,-122.421973,37.775998,276938,2011,08,09,12,00,00,-108.473026,128.026638,167.801125
9,"MALICIOUS MISCHIEF, VANDALISM",Sunday,RICHMOND,NONE,1700 Block of LYON ST,-122.445239,37.786977,146369,2013,06,02,21,00,00,-108.492657,128.051953,167.833129


In [18]:
df=df.drop(labels='rot60_X',axis=1)

df_test=df_test.drop(labels='rot60_X',axis=1)

In [19]:
df=df.drop(labels='rot60_Y',axis=1)

df_test=df_test.drop(labels='rot60_Y',axis=1)

In [20]:
df=df.drop(labels='Second',axis=1)

df_test=df_test.drop(labels='Second',axis=1)

In [21]:
df

,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,Id,Year,Month,Day,Hour,Minute,radial60
0,SEX OFFENSES FORCIBLE,"FORCIBLE RAPE, BODILY FORCE",Friday,MISSION,NONE,2100 Block of MISSION ST,-122.419331,37.762264,141546,2013,06,28,17,40,167.797114
1,LIQUOR LAWS,CONSUMING ALCOHOL IN PUBLIC VIEW,Thursday,SOUTHERN,"ARREST, BOOKED",1000 Block of MARKET ST,-122.411340,37.781271,794152,2004,02,19,02,46,167.786808
2,FRAUD,"CREDIT CARD, THEFT BY USE OF",Wednesday,SOUTHERN,NONE,800 Block of BRYANT ST,-122.403405,37.775421,531205,2007,11,14,00,01,167.775829
3,ROBBERY,ROBBERY OF A CHAIN STORE WITH A GUN,Thursday,BAYVIEW,DISTRICT ATTORNEY REFUSES TO PROSECUTE,2400 Block of SAN BRUNO AV,-122.404715,37.730161,523137,2007,12,27,18,30,167.776249
4,OTHER OFFENSES,PROBATION VIOLATION,Sunday,SOUTHERN,"ARREST, BOOKED",4TH ST / STEVENSON ST,-122.405239,37.785265,200968,2012,09,09,17,02,167.778623
5,WARRANTS,ENROUTE TO DEPARTMENT OF CORRECTIONS,Wednesday,MISSION,"ARREST, BOOKED",15TH ST / NATOMA ST,-122.418405,37.766760,75749,2014,05,14,06,14,167.795988
6,BURGLARY,"BURGLARY, UNLAWFUL ENTRY",Wednesday,CENTRAL,"ARREST, BOOKED",0 Block of MARK LN,-122.404651,37.790442,617521,2006,08,09,23,45,167.777979
7,SUSPICIOUS OCC,SUSPICIOUS OCCURRENCE,Friday,INGLESIDE,NONE,100 Block of PUTNAM ST,-122.410871,37.737526,186953,2012,11,16,12,41,167.784851
8,OTHER OFFENSES,"DRIVERS LICENSE, SUSPENDED OR REVOKED",Monday,BAYVIEW,"ARREST, CITED",HALE ST / SAN BRUNO AV,-122.405911,37.732964,584559,2007,02,05,23,53,167.777961
9,DRUG/NARCOTIC,UNDER THE INFLUENCE OF CONTROLLED SUBSTANCES,Tuesday,NORTHERN,"ARREST, BOOKED",ELLIS ST / LARKIN ST,-122.417710,37.784236,653619,2006,02,07,07,41,167.795570


In [22]:
df_test

,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,Id,Year,Month,Day,Hour,Minute,radial60
0,ATTEMPTED GRAND THEFT FROM PERSON,Sunday,TENDERLOIN,NONE,100 Block of POWELL ST,-122.407878,37.785968,349598,2010,07,04,23,00,167.782237
1,BATTERY OF A POLICE OFFICER,Saturday,PARK,"ARREST, BOOKED",2400 Block of GEARY BL,-122.443597,37.782644,766313,2004,06,26,22,43,167.830764
2,VIOLATION OF MUNICIPAL CODE,Saturday,SOUTHERN,"ARREST, CITED",HOWARD ST / 3RD ST,-122.400474,37.785029,169887,2013,02,09,21,26,167.772129
3,POSS OF PROHIBITED WEAPON,Sunday,NORTHERN,"ARREST, CITED",VANNESS AV / HAYES ST,-122.419698,37.777301,594704,2006,12,03,22,30,167.798067
4,PETTY THEFT FROM UNLOCKED AUTO,Sunday,PARK,NONE,BEULAH ST / SHRADER ST,-122.451488,37.767516,47900,2014,09,21,08,45,167.841050
5,LOST PROPERTY,Sunday,TENDERLOIN,NONE,EDDY ST / JONES ST,-122.412597,37.783932,339260,2010,08,29,18,00,167.788600
6,INFLICT INJURY ON COHABITEE,Sunday,NORTHERN,NONE,100 Block of HAIGHT ST,-122.424641,37.772789,169575,2013,02,10,17,00,167.804660
7,"VEHICLE, RECOVERED, AUTO",Monday,TARAVAL,NONE,TAPIA DR / PINTO AV,-122.481353,37.720399,681483,2005,09,19,16,00,167.880289
8,"CHECKS, MAKE OR PASS FICTITIOUS",Tuesday,NORTHERN,NONE,200 Block of FELL ST,-122.421973,37.775998,276938,2011,08,09,12,00,167.801125
9,"MALICIOUS MISCHIEF, VANDALISM",Sunday,RICHMOND,NONE,1700 Block of LYON ST,-122.445239,37.786977,146369,2013,06,02,21,00,167.833129


# Minute Clubbing

In [23]:
df['Minute']=df['Minute'].apply(lambda x:int(x))
df['Minute']=df['Minute'].apply(lambda x : 'low' if x <31 else 'high')

df_test['Minute']=df_test['Minute'].apply(lambda x:int(x))
df_test['Minute']=df_test['Minute'].apply(lambda x : 'low' if x <31 else 'high')


# DayOfWeek

In [24]:
df['DayOfWeek']= df['DayOfWeek'].apply(lambda x : 'WeekHigh' if x in ('Wednesday','Friday') else ('WeekMed' if x in ('Tuesday','Thursday','Saturday') else 'WeekLow'))


df_test['DayOfWeek']= df_test['DayOfWeek'].apply(lambda x : 'WeekHigh' if x in ('Wednesday','Friday') else ('WeekMed' if x in ('Tuesday','Thursday','Saturday') else 'WeekLow'))


# Address Update

In [25]:
df['Intersection']=df['Address'].apply(lambda x : 1 if '/' in x else 0)
df['Block']=df['Address'].apply(lambda x : 1 if 'Block' in x else 0)
df_test['Intersection']=df_test['Address'].apply(lambda x : 1 if '/' in x else 0)
df_test['Block']=df_test['Address'].apply(lambda x : 1 if 'Block' in x else 0)

In [26]:
address=pd.DataFrame(df['Address'],columns=['Address'])
address=address.Address.str.split(pat=" /",expand=True )

address.columns=['Address','Intr2']

address=address.Address.str.split(pat=" /",expand=True )
address.columns=['Address']

string=address.iloc[:,0]
string=string.str.strip()

address_fram=string.to_frame()
temp=address_fram['Address'].astype(str).str[-2:]

address=temp.to_frame()

address['Address']=address['Address'].apply(lambda x :( x if x in ("ST","AV","LN","DR","BL","HY","CT","RD","PL","PZ","TR","AL","CR","WK","EX","RW") else (("I-80" if x in ("80") else ("HWY" if x in ("WY") else ("WAY" if x in ("AY") else ("TER" if x in ("ER") else ("ALMS" if x in ("MS") else ("MAR" if x in ("AR") else ("PARK" if x in ("RK") else ("STWY" if x in ("WY") else ("VIA" if x in ("NO") else ("BLOCK")))))))))))))
df=df.drop(labels=['Address'],axis=1)
df=pd.concat([address,df],axis=1)



# Address splitting

In [27]:
address_test=pd.DataFrame(df_test['Address'],columns=['Address'])
address_test=address_test.Address.str.split(pat=" /",expand=True )

address_test.columns=['Address','Intr2']

address_test=address_test.Address.str.split(pat=" /",expand=True )
address_test.columns=['Address']

string_test=address_test.iloc[:,0]
string_test=string_test.str.strip()

address_fram_test=string_test.to_frame()
temp_test=address_fram_test['Address'].astype(str).str[-2:]

address_test=temp_test.to_frame()

address_test['Address']=address_test['Address'].apply(lambda x :( x if x in ("ST","AV","LN","DR","BL","HY","CT","RD","PL","PZ","TR","AL","CR","WK","EX","RW") else (("I-80" if x in ("80") else ("HWY" if x in ("WY") else ("WAY" if x in ("AY") else ("TER" if x in ("ER") else ("ALMS" if x in ("MS") else ("MAR" if x in ("AR") else ("PARK" if x in ("RK") else ("STWY" if x in ("WY") else ("VIA" if x in ("NO") else ("BLOCK")))))))))))))
df_test=df_test.drop(labels=['Address'],axis=1)
df_test=pd.concat([address_test,df_test],axis=1)

In [28]:
df

,Address,Category,Descript,DayOfWeek,PdDistrict,Resolution,X,Y,Id,Year,Month,Day,Hour,Minute,radial60,Intersection,Block
0,ST,SEX OFFENSES FORCIBLE,"FORCIBLE RAPE, BODILY FORCE",WeekHigh,MISSION,NONE,-122.419331,37.762264,141546,2013,06,28,17,high,167.797114,0,1
1,ST,LIQUOR LAWS,CONSUMING ALCOHOL IN PUBLIC VIEW,WeekMed,SOUTHERN,"ARREST, BOOKED",-122.411340,37.781271,794152,2004,02,19,02,high,167.786808,0,1
2,ST,FRAUD,"CREDIT CARD, THEFT BY USE OF",WeekHigh,SOUTHERN,NONE,-122.403405,37.775421,531205,2007,11,14,00,low,167.775829,0,1
3,AV,ROBBERY,ROBBERY OF A CHAIN STORE WITH A GUN,WeekMed,BAYVIEW,DISTRICT ATTORNEY REFUSES TO PROSECUTE,-122.404715,37.730161,523137,2007,12,27,18,low,167.776249,0,1
4,ST,OTHER OFFENSES,PROBATION VIOLATION,WeekLow,SOUTHERN,"ARREST, BOOKED",-122.405239,37.785265,200968,2012,09,09,17,low,167.778623,1,0
5,ST,WARRANTS,ENROUTE TO DEPARTMENT OF CORRECTIONS,WeekHigh,MISSION,"ARREST, BOOKED",-122.418405,37.766760,75749,2014,05,14,06,low,167.795988,1,0
6,LN,BURGLARY,"BURGLARY, UNLAWFUL ENTRY",WeekHigh,CENTRAL,"ARREST, BOOKED",-122.404651,37.790442,617521,2006,08,09,23,high,167.777979,0,1
7,ST,SUSPICIOUS OCC,SUSPICIOUS OCCURRENCE,WeekHigh,INGLESIDE,NONE,-122.410871,37.737526,186953,2012,11,16,12,high,167.784851,0,1
8,ST,OTHER OFFENSES,"DRIVERS LICENSE, SUSPENDED OR REVOKED",WeekLow,BAYVIEW,"ARREST, CITED",-122.405911,37.732964,584559,2007,02,05,23,high,167.777961,1,0
9,ST,DRUG/NARCOTIC,UNDER THE INFLUENCE OF CONTROLLED SUBSTANCES,WeekMed,NORTHERN,"ARREST, BOOKED",-122.417710,37.784236,653619,2006,02,07,07,high,167.795570,1,0


In [29]:
df_test

,Address,Descript,DayOfWeek,PdDistrict,Resolution,X,Y,Id,Year,Month,Day,Hour,Minute,radial60,Intersection,Block
0,ST,ATTEMPTED GRAND THEFT FROM PERSON,WeekLow,TENDERLOIN,NONE,-122.407878,37.785968,349598,2010,07,04,23,low,167.782237,0,1
1,BL,BATTERY OF A POLICE OFFICER,WeekMed,PARK,"ARREST, BOOKED",-122.443597,37.782644,766313,2004,06,26,22,high,167.830764,0,1
2,ST,VIOLATION OF MUNICIPAL CODE,WeekMed,SOUTHERN,"ARREST, CITED",-122.400474,37.785029,169887,2013,02,09,21,low,167.772129,1,0
3,AV,POSS OF PROHIBITED WEAPON,WeekLow,NORTHERN,"ARREST, CITED",-122.419698,37.777301,594704,2006,12,03,22,low,167.798067,1,0
4,ST,PETTY THEFT FROM UNLOCKED AUTO,WeekLow,PARK,NONE,-122.451488,37.767516,47900,2014,09,21,08,high,167.841050,1,0
5,ST,LOST PROPERTY,WeekLow,TENDERLOIN,NONE,-122.412597,37.783932,339260,2010,08,29,18,low,167.788600,1,0
6,ST,INFLICT INJURY ON COHABITEE,WeekLow,NORTHERN,NONE,-122.424641,37.772789,169575,2013,02,10,17,low,167.804660,0,1
7,DR,"VEHICLE, RECOVERED, AUTO",WeekLow,TARAVAL,NONE,-122.481353,37.720399,681483,2005,09,19,16,low,167.880289,1,0
8,ST,"CHECKS, MAKE OR PASS FICTITIOUS",WeekMed,NORTHERN,NONE,-122.421973,37.775998,276938,2011,08,09,12,low,167.801125,0,1
9,ST,"MALICIOUS MISCHIEF, VANDALISM",WeekLow,RICHMOND,NONE,-122.445239,37.786977,146369,2013,06,02,21,low,167.833129,0,1


In [30]:
Id=df['Id']
df=df.drop(['Descript','Resolution','Id'],axis=1)

#----------test data---------

Id_test=df_test['Id']
df_test=df_test.drop(['Descript','Resolution','Id'],axis=1)

In [31]:
df_test.columns

Index(['Address', 'DayOfWeek', 'PdDistrict', 'X', 'Y', 'Year', 'Month', 'Day',
       'Hour', 'Minute', 'radial60', 'Intersection', 'Block'],
      dtype='object')

In [32]:
data['PdDistrict'].unique()

array(['MISSION', 'SOUTHERN', 'BAYVIEW', 'CENTRAL', 'INGLESIDE',
       'NORTHERN', 'RICHMOND', 'TARAVAL', 'TENDERLOIN', 'PARK'],
      dtype=object)

# Label Encoding

In [33]:

df.columns

Index(['Address', 'Category', 'DayOfWeek', 'PdDistrict', 'X', 'Y', 'Year',
       'Month', 'Day', 'Hour', 'Minute', 'radial60', 'Intersection', 'Block'],
      dtype='object')

In [34]:
df_test.columns

Index(['Address', 'DayOfWeek', 'PdDistrict', 'X', 'Y', 'Year', 'Month', 'Day',
       'Hour', 'Minute', 'radial60', 'Intersection', 'Block'],
      dtype='object')

In [35]:
le = preprocessing.LabelEncoder()

Label Encoding :DayOfWeek

In [36]:

#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['DayOfWeek'])
Day=pd.DataFrame(le_res)
Day.columns=['DayOfWeek']
df=df.drop(labels=['DayOfWeek'],axis=1)
df=pd.concat([Day,df],axis=1)

#----------test data----------

le_res_test=le.fit_transform(df_test['DayOfWeek'])
Day_test=pd.DataFrame(le_res_test)
Day_test.columns=['DayOfWeek']
df_test=df_test.drop(labels=['DayOfWeek'],axis=1)
df_test=pd.concat([Day_test,df_test],axis=1)

Label Encoding: PdDistrict

In [37]:

le_res=le.fit_transform(df['PdDistrict'])
District=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
District.columns=['District']
df=df.drop(labels=['PdDistrict'],axis=1)
df=pd.concat([District,df],axis=1)


le_res_test=le.fit_transform(df_test['PdDistrict'])
District_test=pd.DataFrame(le_res_test)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
District_test.columns=['District']
df_test=df_test.drop(labels=['PdDistrict'],axis=1)
df_test=pd.concat([District_test,df_test],axis=1)


Label encoding: Year

In [38]:

#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['Year'])
Year=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Year.columns=['Year']
df=df.drop(labels=['Year'],axis=1)
df=pd.concat([Year,df],axis=1)


#le = preprocessing.LabelEncoder()
le_res_test=le.fit_transform(df_test['Year'])
Year_test=pd.DataFrame(le_res_test)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Year_test.columns=['Year']
df_test=df_test.drop(labels=['Year'],axis=1)
df_test=pd.concat([Year_test,df_test],axis=1)


In [39]:
df_test

,Year,District,DayOfWeek,Address,X,Y,Month,Day,Hour,Minute,radial60,Intersection,Block
0,7,9,1,ST,-122.407878,37.785968,07,04,23,low,167.782237,0,1
1,1,5,2,BL,-122.443597,37.782644,06,26,22,high,167.830764,0,1
2,10,7,2,ST,-122.400474,37.785029,02,09,21,low,167.772129,1,0
3,3,4,1,AV,-122.419698,37.777301,12,03,22,low,167.798067,1,0
4,11,5,1,ST,-122.451488,37.767516,09,21,08,high,167.841050,1,0
5,7,9,1,ST,-122.412597,37.783932,08,29,18,low,167.788600,1,0
6,10,4,1,ST,-122.424641,37.772789,02,10,17,low,167.804660,0,1
7,2,8,1,DR,-122.481353,37.720399,09,19,16,low,167.880289,1,0
8,8,4,2,ST,-122.421973,37.775998,08,09,12,low,167.801125,0,1
9,10,6,1,ST,-122.445239,37.786977,06,02,21,low,167.833129,0,1


Label encoding: Month

In [40]:

#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['Month'])
Month=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Month.columns=['Month']
df=df.drop(labels=['Month'],axis=1)
df=pd.concat([Month,df],axis=1)


#le = preprocessing.LabelEncoder()
le_res_test=le.fit_transform(df_test['Month'])
Month_test=pd.DataFrame(le_res_test)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Month_test.columns=['Month']
df_test=df_test.drop(labels=['Month'],axis=1)
df_test=pd.concat([Month_test,df_test],axis=1)


Label encoding: Day

In [41]:

#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['Day'])
Day=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Day.columns=['Day']
df=df.drop(labels=['Day'],axis=1)
df=pd.concat([Day,df],axis=1)


#le = preprocessing.LabelEncoder()
le_res_test=le.fit_transform(df_test['Day'])
Day_test=pd.DataFrame(le_res_test)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Day_test.columns=['Day']
df_test=df_test.drop(labels=['Day'],axis=1)
df_test=pd.concat([Day_test,df_test],axis=1)


Label encoding: Hour

In [42]:

#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['Hour'])
Hour=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Hour.columns=['Hour']
df=df.drop(labels=['Hour'],axis=1)
df=pd.concat([Hour,df],axis=1)


#le = preprocessing.LabelEncoder()
le_res_test=le.fit_transform(df_test['Hour'])
Hour_test=pd.DataFrame(le_res_test)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Hour_test.columns=['Hour']
df_test=df_test.drop(labels=['Hour'],axis=1)
df_test=pd.concat([Hour_test,df_test],axis=1)


Label encoding: Minute

In [43]:

le_res=le.fit_transform(df['Minute'])
Minute=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Minute.columns=['Minute']
df=df.drop(labels=['Minute'],axis=1)
df=pd.concat([Minute,df],axis=1)


le_res_test=le.fit_transform(df_test['Minute'])
Minute_test=pd.DataFrame(le_res_test)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Minute_test.columns=['Minute']
df_test=df_test.drop(labels=['Minute'],axis=1)
df_test=pd.concat([Minute_test,df_test],axis=1)



#label encode street suffix

In [45]:
df_test

,Minute,Hour,Day,Month,Year,District,DayOfWeek,Address,X,Y,radial60,Intersection,Block
0,1,23,3,6,7,9,1,ST,-122.407878,37.785968,167.782237,0,1
1,0,22,25,5,1,5,2,BL,-122.443597,37.782644,167.830764,0,1
2,1,21,8,1,10,7,2,ST,-122.400474,37.785029,167.772129,1,0
3,1,22,2,11,3,4,1,AV,-122.419698,37.777301,167.798067,1,0
4,0,8,20,8,11,5,1,ST,-122.451488,37.767516,167.841050,1,0
5,1,18,28,7,7,9,1,ST,-122.412597,37.783932,167.788600,1,0
6,1,17,9,1,10,4,1,ST,-122.424641,37.772789,167.804660,0,1
7,1,16,18,8,2,8,1,DR,-122.481353,37.720399,167.880289,1,0
8,1,12,8,7,8,4,2,ST,-122.421973,37.775998,167.801125,0,1
9,1,21,1,5,10,6,1,ST,-122.445239,37.786977,167.833129,0,1


In [46]:
le_res=le.fit_transform(df['Category'])
cat=pd.DataFrame(le_res)
cat.columns=['Category']
df=df.drop(labels=['Category'],axis=1)
df=pd.concat([cat,df],axis=1)

df.columns

Index(['Category', 'Minute', 'Hour', 'Day', 'Month', 'Year', 'District',
       'DayOfWeek', 'Address', 'X', 'Y', 'radial60', 'Intersection', 'Block'],
      dtype='object')

In [47]:
df.columns

Index(['Category', 'Minute', 'Hour', 'Day', 'Month', 'Year', 'District',
       'DayOfWeek', 'Address', 'X', 'Y', 'radial60', 'Intersection', 'Block'],
      dtype='object')

In [48]:

le_res=le.fit_transform(df['Address'])
Address=pd.DataFrame(le_res)
Address.columns=['Address']
df=df.drop(labels=['Address'],axis=1)
df=pd.concat([Address,df],axis=1)
le_res=le.fit_transform(df_test['Address'])
Address_test=pd.DataFrame(le_res)
Address_test.columns=['Address']
df_test=df_test.drop(labels=['Address'],axis=1)
df_test=pd.concat([Address_test,df_test],axis=1)

In [49]:
df=df[[ 'District', 'X', 'Y', 'Year','Hour','Address',
       'Month', 'Day', 'radial60', 'Intersection','Block']]


df_test=df_test[[ 'District', 'X', 'Y', 'Year','Hour','Address',
       'Month', 'Day', 'radial60', 'Intersection','Block']]

In [50]:
df_test

,District,X,Y,Year,Hour,Address,Month,Day,radial60,Intersection,Block
0,9,-122.407878,37.785968,7,23,14,6,3,167.782237,0,1
1,5,-122.443597,37.782644,1,22,3,5,25,167.830764,0,1
2,7,-122.400474,37.785029,10,21,14,1,8,167.772129,1,0
3,4,-122.419698,37.777301,3,22,2,11,2,167.798067,1,0
4,5,-122.451488,37.767516,11,8,14,8,20,167.841050,1,0
5,9,-122.412597,37.783932,7,18,14,7,28,167.788600,1,0
6,4,-122.424641,37.772789,10,17,14,1,9,167.804660,0,1
7,8,-122.481353,37.720399,2,16,6,8,18,167.880289,1,0
8,4,-122.421973,37.775998,8,12,14,7,8,167.801125,0,1
9,6,-122.445239,37.786977,10,21,14,5,1,167.833129,0,1


In [ ]:
df['Address'].nunique()

In [ ]:
df_test['Address'].nunique()

In [ ]:
df_test.columns

In [ ]:
df.head()

In [ ]:
""""df=pd.get_dummies(df,columns=[ 'Month',
'District'],drop_first=True)
df=pd.get_dummies(df,columns=[ 'Year'],drop_first=True)
#df_test=pd.get_dummies(df_test,columns=['DayOfWeek','PdDistrict','Year','Month','Day','Hour','Minute'],drop_first=True)
""""



In [ ]:
"""df_test=pd.get_dummies(df_test,columns=[ 'Month',
'District'],drop_first=True)
df_test=pd.get_dummies(df_test,columns=[ 'Year'],drop_first=True)"""

In [ ]:
df.columns.nunique

In [ ]:
#'StreetSuffix_EX','StreetSuffix_RW','StreetSuffix_WK'

X.nunique

In [ ]:
df_test.columns.nunique

In [ ]:
#df=df[['Hour', 'Day', 'Month', 'Year', 'Address',
#       'District','X','radial60','Intersection']]

In [ ]:
#df=pd.get_dummies(df,columns=[ 'Hour'],drop_first=True)



In [ ]:
#df_test=pd.get_dummies(df_test,columns=[ 'Hour'],drop_first=True)

In [ ]:
df.columns.nunique()

In [ ]:
df_test.columns.nunique()

# Applying different Models

In [ ]:
#Independent Column
X=df
X.shape


In [ ]:
X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.40,shuffle=False,random_state=1023)

In [ ]:
#Dependent 
model1= xgb.XGBClassifier(objective='multi:softprob',n_estimators=500,learning_rate =0.2,max_depth=4,min_child_weight=3)
model1.fit(X_train, y_train)
val_pred1=model1.predict(X_val)
#test_pred1=model1.predict(X_test)
val_pred1=pd.DataFrame(val_pred1)
#test_pred1=pd.DataFrame(test_pred1)
val_pred1.columns=['Cat1']
#test_pred1.columns=['Cat1']
#print (log_loss(y_test,y_pred1));

In [ ]:
import lightgbm as lgb
model2= lgb.LGBMClassifier(objective='multiclass',n_estimators=500,learning_rate=0.1,max_depth=4,min_child_samples=21,num_leaves=31)
model2.fit(X_train, y_train)
val_pred2=model2.predict(X_val)
#test_pred2=model2.predict(X_test)
val_pred2=pd.DataFrame(val_pred2)
#test_pred2=pd.DataFrame(test_pred2)
val_pred2.columns=['Cat2']
#test_pred2.columns=['Cat2']

In [ ]:
df_val=np.concatenate((np.array(val_pred1),np.array(val_pred2)),axis=1)
#d_test=np.concatenate((np.array(test_pred1),np.array(test_pred2)),axis=1)

In [ ]:
df_val=pd.DataFrame(df_val)
#d_test=pd.DataFrame(d_test)

In [ ]:
df_val.columns=['Category1','Category2']
df_test.columns=['Category1','Category2']

In [ ]:
model6= xgb.XGBClassifier(objective='multi:softprob')
model6.fit(df_val,df_test)
y_final=model6.predict_proba(df_test)
print (log_loss(y_test,y_final));

# XGBoost Model

In [ ]:
X.head()

In [ ]:
HYPER_PARAMS = {
 
  'learning_rate': 0.2,

 'n_estimators':500,
 'max_depth': 6,
 'subsample': 0.8,
 'colsample_bytree': 0.8,
 'max_delta_step': 1,
 'objective': 'multi:softmax',
 'nthread': 4,
 'seed': 1747
 

 
 

}




model = xgb.XGBClassifier(**HYPER_PARAMS)
model.fit(X,y)



In [ ]:
y_pred=model.predict_proba(df_test)

In [ ]:
temp = data['Category']

le.fit_transform(temp)
le.classes_

In [ ]:
y_pred= pd.DataFrame(y_pred, index=Id_test,columns  = le.classes_)

In [ ]:
y_pred.to_csv("submit.csv", float_format = '%.5F')

In [ ]:
#print (log_loss(y_test,y_pred));

In [ ]:
"""temp = data['Category']

le.fit_transform(temp)
le.classes_
"""

In [ ]:
# y_pred= pd.DataFrame(y_pred, index=Id_test,columns  = le.classes_)



In [ ]:
#from sklearn.linear_model import LogisticRegression
#weight={Address:3,District:3,X:1,Day:2}
#weight={LARCENY/THEFT:35}
#classifier = LogisticRegression(penalty='l1',random_state = 0,class_weight='balanced',multi_class='multinomial', solver='saga',n_jobs=-1)
#classifier = LogisticRegression(random_state=0, penalty='l1',multi_class='multinomial', solver='saga' )
#classifier.fit(X_train[0:50000],y_train[0:50000])

In [ ]:
#y_pred=model.predict_proba(X_test)

In [ ]:
#print (log_loss(y_test,y_pred));

In [ ]:
"""from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(algorithm='auto', leaf_size=60, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')
knn.fit(X_train[0:100000], y_train[0:100000])

"""



In [ ]:
#y_pred=knn.predict_proba(X_test)

In [ ]:
#print (log_loss(y_test,y_pred));